# Example Conversion Efficiency

This example simulates a single electrolyzer stack with a linear ramping power signal. The electrolyzer is initialized as ON. The full structure for an electrolyzer model is defined in the source code, in `modeling_schema.yaml`.

* Conversion is calculated based on the energy consumed per kg of hydrogen. Units are kWh/kg

* Conversion efficiency is calculated by dividing the low-heating value of hydrogen (33.33 kWh/kg) by the actual conversion.

In [37]:
import numpy as np

from electrolyzer import run_electrolyzer
from electrolyzer.inputs.validation import load_modeling_yaml

import matplotlib.pyplot as plt
%matplotlib inline

## Load Electrolyzer configuration from YAML file

This .yaml file initializes a 1.25MW electrolyzer stack as `ON` with the `BaselineDeg` controller. 

The power signal ramps linearly from 10% to 120% of the stack's rated power.

In [40]:
fname_input_modeling = "./modeling_options.yaml"
model_input = load_modeling_yaml(fname_input_modeling)

rated_power = model_input['electrolyzer']['stack']['stack_rating_kW']

# Linear ramp
power_signal_input = np.linspace( 0.1e3 * rated_power , 1.2e3 * rated_power, 1000)

elec_sys, results_df = run_electrolyzer(model_input, power_signal_input)

results_df.tail()

,power_signal,curtailment,kg_rate,stack_1_deg,stack_1_fatigue,stack_1_cycles,stack_1_uptime,stack_1_kg_rate,stack_1_curr_density
995,1.494494e+06,0.244494,0.005224,3.023609e-07,0,0.0,996.0,0.005224,1.790017
996,1.495871e+06,0.245871,0.005224,3.026906e-07,0,0.0,997.0,0.005224,1.790017
997,1.497247e+06,0.247247,0.005224,3.030203e-07,0,0.0,998.0,0.005224,1.790017
998,1.498624e+06,0.248624,0.005224,3.033501e-07,0,0.0,999.0,0.005224,1.790017
999,1.500000e+06,0.250000,0.005224,3.036798e-07,0,0.0,1000.0,0.005224,1.790017


### Co-Plot H2 rate and Conversion Efficiency

* Converts DataFrame data to kW and kg/hr
* low-heating value (LHV) = 33.33 kWh/kg
* Rated power = 1250kW 

Minimum Energy Conversion: 56.15 kW/kg


In [39]:
power_kW = results_df['power_signal'] / 1e3
h2rate_kg_hr = results_df['kg_rate'] * 3600 

lhv = 33.33 # kWh/kg

eff_conv = lhv * h2rate_kg_hr / power_kW

eta_kWh_kg = power_kW / h2rate_kg_hr
print(f"Minimum Energy Conversion: {eta_kWh_kg.min():.2f} kW/kg")

fig, ax = plt.subplots()
ax.set_title('H2 Production and Conversion Efficiency')
ax.plot(power_kW, h2rate_kg_hr, label='H2 Production', color='green')
ax.set_xlabel("Total Power [kW]")
ax.set_ylabel("Mass Flow-Rate [kg/hr]")
ax.vlines(x=rated_power, ymin=0, ymax=1.2*h2rate_kg_hr.max(), linestyle='dashed', color='black', label='Rated-Power')
plt.legend(loc='lower right')

ax2 = ax.twinx()
ax2.plot(power_kW, eff_conv, color='orange', label='Efficiency')
ax2.set_ylabel("Effiency")
plt.legend()
plt.grid()